**Introduction**

This is a Google Colab Notebook which trains a PySlowFast+X3D ensemble model (using pytorchvideo) on a dataset of videos. The dataset's directory should have 3 sub-directories, one for "train", "val", and "test". In each of these directories should be more directories, named with the dataset's classes. Then, within each of these directories should be videos, whose label corresponds to the directory they are placed in (e.g. dataset->train->archery->archery_1.mph).

The "Set-up" section is used for downloading necessary packages and intiliazing variables. Take a look at the variables (such as file paths), and change them accordingly.

The "Class Creation for Dataloaders" section creates the classes/objects necessary for creating and using dataloaders later in the process. Within this section is a commented code block which would change the transform function to one of a resnet. Uncommenting this code block, and changing the model to "make_resnet" in "Model Creation" would use a resnet50 instead.

The "Model Creation" section creates the object that is used as the Lightning Module for the model.

The "Training" section trains the model using the Trainer from Pytorch Lightning.

The "Testing" section tests the model using Trainer.test as well as a script that manually tests every file in the test dataset.


# Set-up


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
!pip install pytorchvideo
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188685 sha256=36b51648007e9840aefda12000a88e0df856bb7474fdc7ac6c256248376c35e2
  Stored in directory: /root/.cache/pip/wheels/ff/4e/81/0f72a543be9ed7eb737c95bfc5da4025e73226b44368074ece
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=0f00dea1bc62e0a2524f543d03746e5e8e846ccc91507ee48f8c2bb465cd6dd4
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created whee

In [ ]:
import os
import pytorch_lightning
import pytorchvideo.data
import torch.utils.data
import torchvision
import pytorchvideo
import pytorchvideo.transforms
from torch.nn.functional import softmax
from typing import Dict
import json
import urllib
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)
import sklearn
#from sklearn.metrics import accuracy_score

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [ ]:
input_dir = '/content/drive/MyDrive/Research/Muscle Video/Datasets/split_workout_videos_v1'
model1_name = "slowfast_r50"
model2_name = "x3d_m"
checkpoint_path = "/content/drive/MyDrive/Research/Muscle Video/Checkpoints/Split/Pretrained/double_backbone/slowfast+x3d"

model1_best_path = '/content/drive/MyDrive/Research/Muscle Video/Checkpoints/Multilabel/slowfast/backbone from exercise/lightning_logs/version_5/checkpoints/epoch=29-step=3854.ckpt'
model2_best_path = '/content/drive/MyDrive/Research/Muscle Video/Checkpoints/Multilabel/x3d_m/backbone/lightning_logs/version_0/checkpoints/epoch=29-step=3900.ckpt'
num_classes=16
num_labels=11
batch_size = 8
num_workers = 4
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4
clip_duration = (num_frames * sampling_rate)/frames_per_second
device=('cuda' if torch.cuda.is_available() else 'cpu')
pretrained=True
learning_rate=0.0001
dropout_rate = 0.6
gamma = 2

model_transform_params  = {
    "x3d_xs": {
        "side_size": 182,
        "crop_size": 182,
        "num_frames": 4,
        "sampling_rate": 12,
    },
    "x3d_s": {
        "side_size": 182,
        "crop_size": 182,
        "num_frames": 13,
        "sampling_rate": 6,
    },
    "x3d_m": {
        "side_size": 256,
        "crop_size": 256,
        "num_frames": 16,
        "sampling_rate": 5,
    }
}
transform_params = model_transform_params[model2_name]

In [ ]:
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

8

In [ ]:
pred_to_class = {
    0: "triceps",
    1: "lats",
    2: "biceps",
    3: "quads",
    4: "glutes",
    5: "shoulders",
    6: "abs",
    7: "obliques",
    8: "chest",
    9: "lower back",
    10: "hamstrings",
}

id_to_exercise = {
    0: "bench press",
    1: "bicep curl",
    2: "chest fly machine",
    3: "deadlift",
    4: "hip thrust",
    5: "lat pulling",
    6: "lateral raise",
    7: "leg extension",
    8: "leg raises",
    9: "push-up",
    10: "russian twist",
    11: "shoulder press",
    12: "squat",
    13: "t bar row",
    14: "tricep Pushdown",
    15: "tricep dips",
}

class_to_label = {
    0: [8, 5, 0],
    1: [2],
    2: [8],
    3: [4, 9, 10],
    4: [4],
    5: [1, 2],
    6: [5],
    7: [3],
    8: [6],
    9: [8, 5, 0],
    10: [6, 7],
    11: [5],
    12: [3, 4, 10],
    13: [1, 2],
    14: [0],
    15: [0],
}

id_to_label = {
    0: [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    1: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    3: [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0],
    4: [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    5: [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    6: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    7: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    8: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
    9: [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    10: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0],
    11: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    12: [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    13: [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    14: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    15: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
}

# Class Creation for Dataloaders

In [ ]:
class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors.
    """
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

slowfast_transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

x3d_transform = ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(transform_params["num_frames"]),
        ]
    ),
)


def post_act(input):
  return softmax(input, dim=1)

'''
from pytorchvideo.models.slowfast import create_slowfast

def make_slowfast():
    return create_slowfast(
        input_channels=(3, 3),
        model_depth=18,
        model_num_class=num_classes,
        norm=nn.BatchNorm3d,
        dropout_rate=dropout_rate,
    )
'''

'\nfrom pytorchvideo.models.slowfast import create_slowfast\n\ndef make_slowfast():\n    return create_slowfast(\n        input_channels=(3, 3),\n        model_depth=18,\n        model_num_class=num_classes,\n        norm=nn.BatchNorm3d,\n        dropout_rate=dropout_rate,\n    )\n'

In [ ]:
from pytorch_lightning.utilities.combined_loader import CombinedLoader

class VideosDataModule(pytorch_lightning.LightningDataModule):

    # Dataset configuration
    _DATA_PATH = input_dir
    _CLIP_DURATION = clip_duration  # Duration of sampled clip for each video
    _BATCH_SIZE = batch_size
    _NUM_WORKERS = num_workers  # Number of parallel processes fetching data

    def train_dataloader(self):
        #Create the train partition from the list of video labels and video paths
        train_dataset = pytorchvideo.data.labeled_video_dataset(
            data_path=os.path.join(self._DATA_PATH, 'train'),
            clip_sampler=pytorchvideo.data.make_clip_sampler("random", self._CLIP_DURATION),
            decode_audio=False,
            transform=slowfast_transform
        )

        return torch.utils.data.DataLoader(
            train_dataset,
            batch_size=self._BATCH_SIZE,
            num_workers=self._NUM_WORKERS
        )

    def val_dataloader(self):
        #Create the validation partition from the list of video labels and video paths
        val_dataset = pytorchvideo.data.labeled_video_dataset(
            data_path=os.path.join(self._DATA_PATH, 'val'),
            clip_sampler=pytorchvideo.data.make_clip_sampler("random", self._CLIP_DURATION),
            decode_audio=False,
            transform=slowfast_transform
        )

        return torch.utils.data.DataLoader(
            val_dataset,
            batch_size=self._BATCH_SIZE,
            num_workers=self._NUM_WORKERS,
        )

    def test_dataloader(self):
        test_dataset = pytorchvideo.data.labeled_video_dataset(
            data_path=os.path.join(self._DATA_PATH, 'test'),
            clip_sampler=pytorchvideo.data.make_clip_sampler("random", self._CLIP_DURATION),
            decode_audio=False,
            transform=slowfast_transform
        )

        test_dataloader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=self._BATCH_SIZE,
            num_workers=self._NUM_WORKERS,
        )

        return test_dataloader

# Model Creation


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchmetrics.classification import MulticlassAccuracy
from torch.nn import Linear
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

mca1 = MulticlassAccuracy(num_classes=num_classes, average='micro', top_k=1)
mca5 = MulticlassAccuracy(num_classes=num_classes, average='micro', top_k=5)

class OG_X3D_VideoClassificationModule(pytorch_lightning.LightningModule):

    def __init__(self):
        super().__init__()

        self.model = torch.hub.load('facebookresearch/pytorchvideo', model2_name, pretrained=True)
        self.model.to(device)
        self.model.blocks[5].proj = Linear(in_features=2048, out_features=num_classes, bias=True)
        self.model.train()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # The model expects a video tensor of shape (B, C, T, H, W), which is the
        # format provided by the dataset
        y_hat = self.model(batch["video"])

        # Compute cross entropy loss, loss.backwards will be called behind the scenes
        # by PyTorchLightning after being returned from this method.

        loss = F.cross_entropy(y_hat, batch["label"])

        # Log the train loss to Tensorboard
        self.log("train_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("train_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("train_accuracy_top_5", acc5.item(), batch_size=batch_size)

        print("train_loss:", loss.item(), "train_accuracy_top_1:", acc1.item(), "train_accuracy_top_5:", acc5.item())

        return loss

    def validation_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        loss = F.cross_entropy(y_hat, batch["label"])

        self.log("val_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("val_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("val_accuracy_top_5", acc5.item(), batch_size=batch_size)

        print("val_loss:", loss.item(), "val_accuracy_top_1:", acc1.item(), "val_accuracy_top_5:", acc5.item())

        return loss

    def configure_optimizers(self):
        """
        Setup the Adam optimizer. Note, that this function also can return a lr scheduler, which is
        usually useful for training video models.
        """
        return torch.optim.Adam(self.parameters(), lr=learning_rate)

    def test_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        loss = F.cross_entropy(y_hat, batch["label"])

        # logs metrics for each testing_step,
        # and the average across the epoch, to the progress bar and logger
        self.log("test_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("test_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("test_accuracy_top_5", acc5.item(), batch_size=batch_size)

        softed = post_act(y_hat)
        auc = roc_auc_score(y_true=batch["label"].cpu(), y_score=softed.cpu(), multi_class='ovr', average='micro', labels=np.arange(num_classes))
        self.log("auc", auc, batch_size=batch_size)

        pred_classes = []
        for x in softed:
          class_index = x.topk(k=1).indices
          class_index = class_index[0]
          pred_classes.append(class_index)
        pred_classes = torch.Tensor(pred_classes)

        rpf1 = precision_recall_fscore_support(y_true=batch["label"].cpu(), y_pred=pred_classes.cpu(), beta=1, labels=np.arange(num_classes), average='macro', zero_division=1)
        precision = rpf1[0]
        recall = rpf1[1]
        f1 = rpf1[2]
        self.log("precision", precision, batch_size=batch_size)
        self.log("recall", recall, batch_size=batch_size)
        self.log("f1", f1, batch_size=batch_size)

        print("test_loss:", loss.item(), "test_accuracy_top_1:", acc1.item(), "test_accuracy_top_5:", acc5.item(), "auc:", auc, "precision:", precision, "recall:", recall, "f1:", f1)

        return loss

x3d_backbone_checkpoint = '/content/drive/MyDrive/Research/Muscle Video/Checkpoints/x3d/x3d_m/lightning_logs/version_3/checkpoints/epoch=29-step=3868.ckpt'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchmetrics.classification import MulticlassAccuracy
from torch.nn import Linear
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

mca1 = MulticlassAccuracy(num_classes=num_classes, average='micro', top_k=1)
mca5 = MulticlassAccuracy(num_classes=num_classes, average='micro', top_k=5)

class x3d_VideoClassificationLightningModule(pytorch_lightning.LightningModule):

    def __init__(self):
        super().__init__()

        self.model = OG_X3D_VideoClassificationModule.load_from_checkpoint(x3d_backbone_checkpoint)
        self.model.to(device)

        for param in self.model.parameters():
          param.requires_grad = False

        self.model.model.blocks[5].proj = nn.Linear(in_features=2048, out_features=11, bias=True)

        for param in self.model.model.blocks[5].proj.parameters():
          param.requires_grad = True

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # The model expects a video tensor of shape (B, C, T, H, W), which is the
        # format provided by the dataset
        y_hat = self.model(batch["video"])

        # Compute cross entropy loss, loss.backwards will be called behind the scenes
        # by PyTorchLightning after being returned from this method.

        loss = F.cross_entropy(y_hat, batch["label"])

        # Log the train loss to Tensorboard
        self.log("train_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("train_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("train_accuracy_top_5", acc5.item(), batch_size=batch_size)

        print("train_loss:", loss.item(), "train_accuracy_top_1:", acc1.item(), "train_accuracy_top_5:", acc5.item())

        return loss

    def validation_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        loss = F.cross_entropy(y_hat, batch["label"])

        self.log("val_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("val_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("val_accuracy_top_5", acc5.item(), batch_size=batch_size)

        print("val_loss:", loss.item(), "val_accuracy_top_1:", acc1.item(), "val_accuracy_top_5:", acc5.item())

        return loss

    def configure_optimizers(self):
        """
        Setup the Adam optimizer. Note, that this function also can return a lr scheduler, which is
        usually useful for training video models.
        """
        return torch.optim.Adam(self.parameters(), lr=learning_rate)

    def test_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        loss = F.cross_entropy(y_hat, batch["label"])

        # logs metrics for each testing_step,
        # and the average across the epoch, to the progress bar and logger
        self.log("test_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("test_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("test_accuracy_top_5", acc5.item(), batch_size=batch_size)

        softed = post_act(y_hat)
        auc = roc_auc_score(y_true=batch["label"].cpu(), y_score=softed.cpu(), multi_class='ovr', average='micro', labels=np.arange(num_classes))
        self.log("auc", auc, batch_size=batch_size)

        pred_classes = []
        for x in softed:
          class_index = x.topk(k=1).indices
          class_index = class_index[0]
          pred_classes.append(class_index)
        pred_classes = torch.Tensor(pred_classes)

        rpf1 = precision_recall_fscore_support(y_true=batch["label"].cpu(), y_pred=pred_classes.cpu(), beta=1, labels=np.arange(num_classes), average='macro', zero_division=1)
        precision = rpf1[0]
        recall = rpf1[1]
        f1 = rpf1[2]
        self.log("precision", precision, batch_size=batch_size)
        self.log("recall", recall, batch_size=batch_size)
        self.log("f1", f1, batch_size=batch_size)

        print("test_loss:", loss.item(), "test_accuracy_top_1:", acc1.item(), "test_accuracy_top_5:", acc5.item(), "auc:", auc, "precision:", precision, "recall:", recall, "f1:", f1)

        return loss

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchmetrics.classification import MulticlassAccuracy

mca1 = MulticlassAccuracy(num_classes=num_classes, average='micro', top_k=1)
mca5 = MulticlassAccuracy(num_classes=num_classes, average='micro', top_k=5)

class OG_VideoClassificationModule(pytorch_lightning.LightningModule):

    def __init__(self):
        super().__init__()

        self.model = torch.hub.load("facebookresearch/pytorchvideo", model=model1_name, pretrained=True)
        self.model.to(device)
        self.model.blocks[6].proj = nn.Linear(in_features=2304, out_features=16, bias=True)
        self.model.train()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # The model expects a video tensor of shape (B, C, T, H, W), which is the
        # format provided by the dataset
        y_hat = self.model(batch["video"])

        # Compute cross entropy loss, loss.backwards will be called behind the scenes
        # by PyTorchLightning after being returned from this method.

        loss = F.cross_entropy(y_hat, batch["label"])

        # Log the train loss to Tensorboard
        self.log("train_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("train_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("train_accuracy_top_5", acc5.item(), batch_size=batch_size)

        #last_layer = self.model.blocks[6].proj
        #embeddings = []

        print("train_loss:", loss.item(), "train_accuracy_top_1:", acc1.item(), "train_accuracy_top_5:", acc5.item())

        return loss

    def validation_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        loss = F.cross_entropy(y_hat, batch["label"])

        self.log("val_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("val_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("val_accuracy_top_5", acc5.item(), batch_size=batch_size)

        print("val_loss:", loss.item(), "val_accuracy_top_1:", acc1.item(), "val_accuracy_top_5:", acc5.item())

        return loss

    def configure_optimizers(self):
        """
        Setup the Adam optimizer. Note, that this function also can return a lr scheduler, which is
        usually useful for training video models.
        """
        return torch.optim.Adam(self.parameters(), lr=learning_rate)

    def test_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        loss = F.cross_entropy(y_hat, batch["label"])

        # logs metrics for each testing_step,
        # and the average across the epoch, to the progress bar and logger
        self.log("test_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("test_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("test_accuracy_top_5", acc5.item(), batch_size=batch_size)

        print("test_loss:", loss.item(), "test_accuracy_top_1:", acc1.item(), "test_accuracy_top_5:", acc5.item())

        return loss

backbone_checkpoint = '/content/drive/MyDrive/Research/Muscle Video/Checkpoints/Split/Pretrained/slowfast_r50/lightning_logs/version_4/checkpoints/epoch=4-step=640.ckpt'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchmetrics.classification import MulticlassAccuracy
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

mca1 = MulticlassAccuracy(num_classes=num_classes, average='micro', top_k=1)
mca5 = MulticlassAccuracy(num_classes=num_classes, average='micro', top_k=5)

class slowfast_VideoClassificationLightningModule(pytorch_lightning.LightningModule):

    def __init__(self):
        super().__init__()

        self.model = OG_VideoClassificationModule.load_from_checkpoint(backbone_checkpoint)
        self.model.to(device)

        for param in self.model.parameters():
          param.requires_grad = False

        self.model.model.blocks[6].proj = nn.Linear(in_features=2304, out_features=11, bias=True)

        for param in self.model.model.blocks[6].proj.parameters():
          param.requires_grad = True

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # The model expects a video tensor of shape (B, C, T, H, W), which is the
        # format provided by the dataset
        y_hat = self.model(batch["video"])

        # Compute cross entropy loss, loss.backwards will be called behind the scenes
        # by PyTorchLightning after being returned from this method.

        loss = F.cross_entropy(y_hat, batch["label"])

        # Log the train loss to Tensorboard
        self.log("train_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("train_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("train_accuracy_top_5", acc5.item(), batch_size=batch_size)

        #last_layer = self.model.blocks[6].proj
        #embeddings = []

        print("train_loss:", loss.item(), "train_accuracy_top_1:", acc1.item(), "train_accuracy_top_5:", acc5.item())

        return loss

    def validation_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        loss = F.cross_entropy(y_hat, batch["label"])

        self.log("val_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("val_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("val_accuracy_top_5", acc5.item(), batch_size=batch_size)

        print("val_loss:", loss.item(), "val_accuracy_top_1:", acc1.item(), "val_accuracy_top_5:", acc5.item())

        return loss

    def configure_optimizers(self):
        """
        Setup the Adam optimizer. Note, that this function also can return a lr scheduler, which is
        usually useful for training video models.
        """
        return torch.optim.Adam(self.parameters(), lr=learning_rate)

    def test_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        loss = F.cross_entropy(y_hat, batch["label"])

        # logs metrics for each testing_step,
        # and the average across the epoch, to the progress bar and logger
        self.log("test_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("test_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("test_accuracy_top_5", acc5.item(), batch_size=batch_size)

        softed = post_act(y_hat)
        auc = roc_auc_score(y_true=batch["label"].cpu(), y_score=softed.cpu(), multi_class='ovr', average='micro', labels=np.arange(num_classes))
        self.log("auc", auc, batch_size=batch_size)

        pred_classes = []
        for x in softed:
          class_index = x.topk(k=1).indices
          class_index = class_index[0]
          pred_classes.append(class_index)
        pred_classes = torch.Tensor(pred_classes)

        rpf1 = precision_recall_fscore_support(y_true=batch["label"].cpu(), y_pred=pred_classes.cpu(), beta=1, labels=np.arange(num_classes), average='macro', zero_division=1)
        precision = rpf1[0]
        recall = rpf1[1]
        f1 = rpf1[2]
        self.log("precision", precision, batch_size=batch_size)
        self.log("recall", recall, batch_size=batch_size)
        self.log("f1", f1, batch_size=batch_size)

        print("test_loss:", loss.item(), "test_accuracy_top_1:", acc1.item(), "test_accuracy_top_5:", acc5.item(), "auc:", auc, "precision:", precision, "recall:", recall, "f1:", f1)

        return loss

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchmetrics.classification import MultilabelAccuracy, MultilabelAUROC, MultilabelRecall, MultilabelPrecision, MultilabelF1Score

mcaM2 = MultilabelAccuracy(num_labels=num_labels, average='micro')
mcaS = MultilabelAccuracy(num_labels=num_labels, average='none')
roc_auc = MultilabelAUROC(num_labels=num_labels, average="micro")
pre = MultilabelPrecision(num_labels=num_labels, average='macro', threshold=0.5)
rec = MultilabelRecall(num_labels=num_labels, average='macro', threshold=0.5)
f1score = MultilabelF1Score(num_labels=num_labels, average='macro', threshold=0.5)

class VideoClassificationLightningModule(pytorch_lightning.LightningModule):

    def __init__(self):
        super().__init__()

        self.model1 = slowfast_VideoClassificationLightningModule.load_from_checkpoint(model1_best_path)
        self.model1.to(device)
        self.model1.eval()

        self.model2 = x3d_VideoClassificationLightningModule.load_from_checkpoint(model2_best_path)
        self.model2.model.model.blocks[5].activation = None
        self.model2.to(device)
        self.model2.eval()
        #print(self.model2)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # The model expects a video tensor of shape (B, C, T, H, W), which is the
        # format provided by the dataset
        y_hat = self.model(batch["video"])

        # Compute cross entropy loss, loss.backwards will be called behind the scenes
        # by PyTorchLightning after being returned from this method.

        loss = F.cross_entropy(y_hat, batch["label"])

        # Log the train loss to Tensorboard
        self.log("train_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("train_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("train_accuracy_top_5", acc5.item(), batch_size=batch_size)

        #last_layer = self.model.blocks[6].proj
        #embeddings = []

        print("train_loss:", loss.item(), "train_accuracy_top_1:", acc1.item(), "train_accuracy_top_5:", acc5.item())

        return loss

    def validation_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        loss = F.cross_entropy(y_hat, batch["label"])

        self.log("val_loss", loss.item(), batch_size=batch_size)

        acc1 = mca1(y_hat.cpu(), batch["label"].cpu())
        acc5 = mca5(y_hat.cpu(), batch["label"].cpu())
        self.log("val_accuracy_top_1", acc1.item(), batch_size=batch_size)
        self.log("val_accuracy_top_5", acc5.item(), batch_size=batch_size)

        print("val_loss:", loss.item(), "val_accuracy_top_1:", acc1.item(), "val_accuracy_top_5:", acc5.item())

        return loss

    def configure_optimizers(self):
        """
        Setup the Adam optimizer. Note, that this function also can return a lr scheduler, which is
        usually useful for training video models.
        """
        return torch.optim.Adam(self.parameters(), lr=learning_rate)

    def test_step(self, batch, batch_idx):

        slowfast_data = batch["video"]
        x3d_data = batch["video"][1]

        y_hat1 = self.model1(slowfast_data)
        y_hat2 = self.model2(x3d_data)

        y_hat = []
        for x in range(0, y_hat1.shape[0]):
          temp = []
          for y in range(0, 11):
            val = 0.75*y_hat1[x][y] + 0.25*y_hat2[x][y]
            temp.append(val)
          y_hat.append(temp)

        y_hat = torch.Tensor(y_hat)

        new_label = torch.tensor([id_to_label[i.item()] for i in batch["label"]])

        loss = F.cross_entropy(y_hat.cpu(), new_label.cpu())
        #loss = torch.Tensor(0)

        # logs metrics for each testing_step,
        # and the average across the epoch, to the progress bar and logger
        self.log("test_loss", loss.item(), batch_size=batch_size)

        accM2 = mcaM2(y_hat.cpu(), new_label.cpu())
        self.log("test_accuracy_micro", accM2.item(), batch_size=batch_size)

        new_label = new_label.type(torch.IntTensor)

        auc = roc_auc(y_hat.cpu(), new_label.cpu())
        self.log("auc", auc.item(), batch_size=batch_size)

        precision = pre(y_hat.cpu(), new_label.cpu())
        recall = rec(y_hat.cpu(), new_label.cpu())
        f1 = f1score(y_hat.cpu(), new_label.cpu())

        self.log("precision", precision.item(), batch_size=batch_size)
        self.log("recall", recall.item(), batch_size=batch_size)
        self.log("f1", f1.item(), batch_size=batch_size)

        print("test_loss:", loss.item(), "test_accuracy_micro:", accM2.item(), "auc:", auc.item(), "precision:", precision.item(), "recall", recall.item(), "f1", f1.item())

        return loss

# Training

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping

early_stopping_callbacks = EarlyStopping(monitor="val_loss", min_delta=0, patience=10, verbose=True, mode="min")

In [ ]:
classification_module = VideoClassificationLightningModule()
data_module = VideosDataModule()
trainer = pytorch_lightning.Trainer(
    default_root_dir=checkpoint_path,
    max_epochs=30,
    accelerator="auto",
    devices="auto",
    strategy='auto',
    enable_checkpointing=True,
    logger=True,
)

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
torch.set_float32_matmul_precision('medium')

In [ ]:
trainer.fit(classification_module, data_module, ckpt_path='/content/drive/MyDrive/Research/Muscle Video/Checkpoints/Split/Pretrained/slowfast_r101/lightning_logs/version_5/checkpoints/epoch=28-step=3754.ckpt')

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/Research/Muscle Video/Checkpoints/Split/Pretrained/slowfast_r101/lightning_logs/version_5/checkpoints/epoch=28-step=3754.ckpt


RuntimeError: ignored

# Testing


In [ ]:
trainer.test(model=classification_module, datamodule=data_module, verbose=True)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

test_loss: 2.013576030731201 test_accuracy_micro: 1.0 auc: 1.0 precision: 0.7272727489471436 recall 0.7272727489471436 f1 0.7272727489471436
test_loss: 2.1450650691986084 test_accuracy_micro: 1.0 auc: 1.0 precision: 0.6363636255264282 recall 0.6363636255264282 f1 0.6363636255264282
test_loss: 0.9566322565078735 test_accuracy_micro: 1.0 auc: 1.0 precision: 0.6363636255264282 recall 0.6363636255264282 f1 0.6363636255264282
test_loss: 3.4235424995422363 test_accuracy_micro: 0.9318181872367859 auc: 0.9611110687255859 precision: 0.6818182468414307 recall 0.575757622718811 f1 0.6151515245437622
test_loss: 1.8996312618255615 test_accuracy_micro: 1.0 auc: 1.0 precision: 0.9090909957885742 recall 0.9090909957885742 f1 0.9090909957885742
test_loss: 0.9732237458229065 test_accuracy_micro: 1.0 auc: 1.0 precision: 0.7272727489471436 recall 0.7272727489471436 f1 0.7272727489471436
test_loss: 1.738102674484253 test_accuracy_micro: 1.0 auc: 0.9999999403953552 precision: 0.5454545617103577 recall 0.545

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│            auc            │    0.9927492141723633     │
│            f1             │    0.6293193101882935     │
│         precision         │    0.6518596410751343     │
│          recall           │     0.618973970413208     │
│    test_accuracy_micro    │    0.9803718328475952     │
│         test_loss         │    1.8453432321548462     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.8453432321548462,
  'test_accuracy_micro': 0.9803718328475952,
  'auc': 0.9927492141723633,
  'precision': 0.6518596410751343,
  'recall': 0.618973970413208,
  'f1': 0.6293193101882935}]